In [1]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import seaborn as sns
import matplotlib.pyplot as plt

# --- 1. CONFIGURACIÓN DE PARÁMETROS (DEBES EDITAR ESTO) ---

# Tiempos de estímulo (en segundos)
# DEBES ajustar esto al tiempo exacto en que encendiste la bobina
STIMULUS_TIME_EXPERIMENT = 300.0  # Ej: 300s = 5 minutos
# Para los controles, usamos un tiempo "ficticio" para validar el método
STIMULUS_TIME_CONTROL = 300.0

# Ventanas de análisis (relativas al tiempo de estímulo)
# (inicio, fin) en segundos.
# PRE_WINDOW: Ventana "antes" del estímulo para el score. Ej: 1 minuto antes.
PRE_WINDOW = (-60.0, 0.0)
# POST_WINDOW: Ventana "después" para medir el efecto.
# Esto debe capturar el DELAY. Ej: (30s, 150s) mide el promedio
# entre 30s y 150s DESPUÉS del estímulo.
POST_WINDOW = (30.0, 150.0)

# Ubicación de los archivos
CONTROL_DIR = './experimentos/de 1000/'
EXPERIMENT_DIR = './experimentos/'
# Extensión de tus archivos de datos. 
# Ajusta el patrón para que coincida con tus archivos, ej: '*.txt', '*.csv', 'exp_*.dat'
# Usaré un patrón genérico ('*') pero sé específico si hay otros archivos
CONTROL_FILE_PATTERN = os.path.join(CONTROL_DIR, '*.txt')
EXPERIMENT_FILE_PATTERN = os.path.join(EXPERIMENT_DIR, '*.txt')

# Opciones
ENABLE_PLOTTING = True  # True para generar un gráfico de diagnóstico por cada archivo
PLOT_DIR = './plots/'  # Directorio para guardar los gráficos

# --- 2. FUNCIÓN PARA EXTRAER FRECUENCIA (DEBES EDITAR ESTO) ---

def extract_frequency_from_filename(filename):
    """
    Extrae la frecuencia del nombre del archivo.
    ESTA ES UNA FUNCIÓN DE EJEMPLO. DEBES ADAPTARLA.
    
    Ej: Si tu archivo se llama 'exp_150Hz_run1.txt',
        queremos que esta función devuelva '150Hz'.
    
    Intenta ser consistente para que agrupe bien (ej: '100Hz', '150Hz')
    """
    import re
    # Ejemplo básico: busca un número seguido de "Hz"
    # El patrón r'(\d+Hz)' busca uno o más dígitos (\d+) seguidos de "Hz"
    match = re.search(r'(\d+Hz)', filename)
    if match:
        return match.group(1)
    
    # Ejemplo 2: si el nombre es la frecuencia (ej: '150.txt')
    # match = re.search(r'(\d+)\.', os.path.basename(filename))
    # if match:
    #     return f"{match.group(1)}Hz"

    # Si no se encuentra, devuelve un 'default' para agruparlo
    return 'unknown_freq'

# --- 3. FUNCIONES DE PROCESAMIENTO Y ANÁLISIS ---

def clean_column_names(df):
    """Limpia los nombres de las columnas para un uso fácil."""
    # Renombra columnas quitando corchetes y unidades
    df = df.rename(columns={
        'Time[s]': 'Time',
        'CO2_smooth[ppm]': 'CO2',
        'dCO2_dt[ppm/s]': 'dCO2_dt',
        'd2CO2_dt2[ppm/s^2]': 'd2CO2_dt2'
    })
    # Si la renombración falla, intenta limpiar los nombres existentes
    if 'Time' not in df.columns:
        df.columns = df.columns.str.replace(r'\[.*\]', '', regex=True).str.strip()
    return df

def process_experiment(filepath, stimulus_time):
    """
    Carga un archivo, calcula la regresión pre-estímulo y los residuales.
    """
    try:
        # Carga el archivo, asumiendo delimitado por espacios
        df = pd.read_csv(filepath, sep='\s+', na_values='nan')
        df = clean_column_names(df)
        
        # Asegurarse de que las columnas 'Time' y 'CO2' existen
        if 'Time' not in df.columns or 'CO2' not in df.columns:
            print(f"Advertencia: Saltando archivo {filepath} - Faltan columnas 'Time' o 'CO2'.")
            return None, 0, 0
            
        # Quitar filas con NaN en Time o CO2, que arruinan la regresión
        df_clean = df.dropna(subset=['Time', 'CO2']).copy()
        if df_clean.empty:
            print(f"Advertencia: Saltando archivo {filepath} - No hay datos válidos (no-NaN).")
            return None, 0, 0

        # 1. Identificar datos Pre-Estímulo
        pre_stim_data = df_clean[df_clean['Time'] < stimulus_time]
        
        if pre_stim_data.shape[0] < 2:
            print(f"Advertencia: Saltando archivo {filepath} - No hay suficientes datos pre-estímulo (<2) para regresión.")
            return None, 0, 0

        # 2. Ajustar regresión lineal (grado 1)
        # m = pendiente, b = intercepto
        m, b = np.polyfit(pre_stim_data['Time'], pre_stim_data['CO2'], 1)

        # 3. Crear columnas de 'Fit' y 'Residual' PARA TODO el dataset
        df_clean['CO2_linear_fit'] = m * df_clean['Time'] + b
        df_clean['CO2_residual'] = df_clean['CO2'] - df_clean['CO2_linear_fit']
        
        return df_clean, m, b

    except Exception as e:
        print(f"Error procesando archivo {filepath}: {e}")
        return None, 0, 0

def calculate_effect_score(df, pre_window, post_window, stimulus_time):
    """
    Calcula el 'Efecto_Neto' comparando el residual promedio
    en la ventana post-estímulo vs. pre-estímulo.
    """
    if df is None:
        return np.nan

    # Definir ventanas absolutas
    pre_start = stimulus_time + pre_window[0]
    pre_end = stimulus_time + pre_window[1]
    post_start = stimulus_time + post_window[0]
    post_end = stimulus_time + post_window[1]

    # Calcular promedios en ventanas
    avg_pre = df[(df['Time'] >= pre_start) & (df['Time'] < pre_end)]['CO2_residual'].mean()
    avg_post = df[(df['Time'] >= post_start) & (df['Time'] < post_end)]['CO2_residual'].mean()

    # Chequear si alguna ventana estuvo vacía (da NaN)
    if pd.isna(avg_pre) or pd.isna(avg_post):
        return np.nan
        
    effect_score = avg_post - avg_pre
    return effect_score

def plot_detrending(df, filename, stimulus_time, plot_savename):
    """
    Genera y guarda un gráfico de diagnóstico (Original vs Residual).
    """
    if df is None:
        return

    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
    
    # 1. Gráfico de datos originales + ajuste lineal
    ax1.plot(df['Time'], df['CO2'], label='CO2 Original (smooth)', alpha=0.8)
    ax1.plot(df['Time'], df['CO2_linear_fit'], label=f'Ajuste lineal pre-estímulo (m={df["slope_m"].iloc[0]:.2f})', linestyle='--', color='k')
    ax1.axvline(x=stimulus_time, color='r', linestyle='--', label='Estímulo')
    ax1.set_title(f'Datos Originales y Ajuste - {filename}')
    ax1.set_ylabel('CO2 [ppm]')
    ax1.legend()
    ax1.grid(True)

    # 2. Gráfico de residuales (el "efecto")
    ax2.plot(df['Time'], df['CO2_residual'], label='CO2 Residual (Efecto)', color='green')
    ax2.axvline(x=stimulus_time, color='r', linestyle='--', label='Estímulo')
    ax2.axhline(y=0, color='k', linestyle='-', linewidth=0.5)
    
    # Sombrear las ventanas de análisis
    pre_start = stimulus_time + PRE_WINDOW[0]
    pre_end = stimulus_time + PRE_WINDOW[1]
    post_start = stimulus_time + POST_WINDOW[0]
    post_end = stimulus_time + POST_WINDOW[1]
    ax2.axvspan(pre_start, pre_end, color='blue', alpha=0.2, label='Ventana Pre-Score')
    ax2.axvspan(post_start, post_end, color='orange', alpha=0.2, label='Ventana Post-Score')
    
    ax2.set_title(f'Residual "Detrended" - Efecto Neto: {df["effect_score"].iloc[0]:.2f}')
    ax2.set_xlabel('Tiempo [s]')
    ax2.set_ylabel('CO2 Residual [ppm]')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.savefig(plot_savename)
    plt.close(fig)


def plot_summary_boxplot(df):
    """
    Genera un boxplot de los 'effect_score' agrupados por frecuencia.
    """
    if df.empty or 'freq' not in df.columns or 'effect_score' not in df.columns:
        print("No se pudo generar el boxplot, datos insuficientes.")
        return

    # Ordenar las frecuencias (opcional, pero se ve mejor)
    # Intenta ordenar numéricamente si es posible, si no, alfabéticamente
    try:
        # Intenta extraer números de las etiquetas 'Hz' para ordenar
        freq_order = sorted(df['freq'].unique(), 
                            key=lambda x: (x != 'control', float(''.join(filter(str.isdigit, x))) if any(char.isdigit() for char in x) else float('inf')))
    except:
        freq_order = sorted(df['freq'].unique())

    plt.figure(figsize=(15, 8))
    
    # Crear el Boxplot
    sns.boxplot(x='freq', y='effect_score', data=df, order=freq_order, palette='Set2')
    
    # Añadir 'swarmplot' para ver los puntos individuales encima
    sns.swarmplot(x='freq', y='effect_score', data=df, order=freq_order, color='0.25')
    
    # Línea de "No Efecto"
    plt.axhline(y=0, color='red', linestyle='--', label='Sin Efecto (0.0)')
    
    plt.title('Comparación de "Score de Efecto" por Frecuencia vs. Control', fontsize=16)
    plt.xlabel('Condición (Frecuencia)', fontsize=12)
    plt.ylabel('Score de Efecto (CO2 Residual Promedio)', fontsize=12)
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    
    # Guardar el gráfico
    plot_filename = 'summary_boxplot.png'
    plt.savefig(plot_filename)
    print(f"\nGráfico resumen guardado en: {plot_filename}")
    plt.close()

# --- 4. SCRIPT PRINCIPAL DE EJECUCIÓN ---

def main():
    print("Iniciando análisis de experimentos...")
    all_results = []
    
    if ENABLE_PLOTTING and not os.path.exists(PLOT_DIR):
        os.makedirs(PLOT_DIR)

    # --- Procesar Controles ---
    print(f"Procesando archivos de control en {CONTROL_DIR}...")
    control_files = glob.glob(CONTROL_FILE_PATTERN)
    if not control_files:
        print("Advertencia: No se encontraron archivos de control.")
        
    for f in control_files:
        basename = os.path.basename(f)
        if os.path.isdir(f): # Evitar procesar directorios
            continue
            
        print(f"  > Procesando control: {basename}")
        processed_df, m, b = process_experiment(f, STIMULUS_TIME_CONTROL)
        
        if processed_df is not None:
            score = calculate_effect_score(processed_df, PRE_WINDOW, POST_WINDOW, STIMULUS_TIME_CONTROL)
            all_results.append({
                'filename': basename,
                'type': 'control',
                'effect_score': score,
                'freq': 'control', # Frecuencia 'control' para agrupar
                'slope_m': m
            })
            
            if ENABLE_PLOTTING:
                # Añadir datos al df para el ploteo
                processed_df['slope_m'] = m
                processed_df['effect_score'] = score
                plot_name = os.path.join(PLOT_DIR, f"plot_CONTROL_{basename}.png")
                plot_detrending(processed_df, basename, STIMULUS_TIME_CONTROL, plot_name)

    # --- Procesar Experimentos ---
    print(f"\nProcesando archivos de experimento en {EXPERIMENT_DIR}...")
    experiment_files = glob.glob(EXPERIMENT_FILE_PATTERN)
    if not experiment_files:
        print("Advertencia: No se encontraron archivos de experimento.")

    for f in experiment_files:
        basename = os.path.basename(f)
        if os.path.isdir(f): # Evitar procesar directorios
            continue
            
        print(f"  > Procesando experimento: {basename}")
        
        # Extraer frecuencia
        freq = extract_frequency_from_filename(f) # Pasar 'f' (ruta completa) o 'basename'
        
        processed_df, m, b = process_experiment(f, STIMULUS_TIME_EXPERIMENT)
        
        if processed_df is not None:
            score = calculate_effect_score(processed_df, PRE_WINDOW, POST_WINDOW, STIMULUS_TIME_EXPERIMENT)
            all_results.append({
                'filename': basename,
                'type': 'experiment',
                'effect_score': score,
                'freq': freq,
                'slope_m': m
            })
            
            if ENABLE_PLOTTING:
                # Añadir datos al df para el ploteo
                processed_df['slope_m'] = m
                processed_df['effect_score'] = score
                plot_name = os.path.join(PLOT_DIR, f"plot_EXP_{basename}.png")
                plot_detrending(processed_df, basename, STIMULUS_TIME_EXPERIMENT, plot_name)

    if not all_results:
        print("\nError: No se pudieron procesar datos. Revisa tus paths y la función 'extract_frequency'. Finalizando.")
        return

    # --- 5. COMPILAR RESULTADOS Y GUARDAR ---
    
    print("\n--- Compilando Resultados ---")
    results_df = pd.DataFrame(all_results)
    
    # Limpiar scores nulos (de experimentos fallidos)
    results_df_clean = results_df.dropna(subset=['effect_score']).copy()
    
    if results_df_clean.empty:
        print("Error: Se procesaron archivos pero no se pudieron calcular 'effect_scores'. Revisa tus ventanas (WINDOW) y datos.")
        return

    # Guardar resultados en un CSV
    results_df_clean.to_csv('analysis_results.csv', index=False)
    
    print("Resultados guardados en 'analysis_results.csv'")
    print(f"Total de {len(results_df_clean)} experimentos procesados exitosamente.")
    
    print("\nResumen de 'Effect Score' por grupo:")
    print(results_df_clean.groupby('freq')['effect_score'].describe())

    # --- 6. ANÁLISIS ESTADÍSTICO (BÁSICO) ---
    
    print("\n--- Análisis Estadístico (Ejemplos) ---")
    
    control_scores = results_df_clean[results_df_clean['type'] == 'control']['effect_score']
    
    if control_scores.empty:
        print("No hay datos de control para estadísticas, se omiten T-tests.")
        return

    # --- Opción A: T-tests (Control vs. Cada Frecuencia) ---
    print("\nOpción A: T-tests (Control vs. Frecuencias)")
    
    freqs_to_test = results_df_clean[results_df_clean['type'] == 'experiment']['freq'].unique()
    
    for freq in freqs_to_test:
        exp_scores = results_df_clean[results_df_clean['freq'] == freq]['effect_score']
        
        if len(exp_scores) < 2 or len(control_scores) < 2:
            print(f"  > {freq}: No hay suficientes datos para T-test.")
            continue
            
        # T-test de Welch (equal_var=False) es más robusto
        t_stat, p_value = ttest_ind(control_scores, exp_scores, equal_var=False)
        print(f"  > Control vs. {freq}:")
        print(f"    P-value = {p_value:.4f} (T-stat = {t_stat:.2f})")
        if p_value < 0.05:
            print("    -> ¡Diferencia estadísticamente significativa!")

    # --- Opción B: ANOVA (Todas las condiciones juntas) ---
    print("\nOpción B: ANOVA (Comparación de todos los grupos)")
    
    if results_df_clean['freq'].nunique() < 2:
        print("  > No hay suficientes grupos 'freq' distintos para ANOVA.")
        return
        
    try:
        # C(freq) trata 'freq' como una variable categórica
        model = ols('effect_score ~ C(freq)', data=results_df_clean).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        
        print(anova_table)
        
        # Test Post-Hoc de Tukey (si el ANOVA es significativo)
        p_anova = anova_table['PR(>F)'][0]
        if p_anova < 0.05:
            print("\n  -> Test Post-Hoc de Tukey (ANOVA fue significativo):")
            tukey = pairwise_tukeyhsd(endog=results_df_clean['effect_score'],
                                      groups=results_df_clean['freq'],
                                      alpha=0.05)
            print(tukey)
        else:
            print("\n  -> ANOVA no fue significativo (p > 0.05), no se realiza test de Tukey.")


    except Exception as e:
        print(f"Error durante ANOVA: {e}")

    # --- 7. GENERAR GRÁFICO RESUMEN (NUEVO PASO) ---
    print("\n--- Generando Gráfico Resumen ---")
    plot_summary_boxplot(results_df_clean)

# --- Ejecutar el script ---
if __name__ == "__main__":
    main()

<>:92: SyntaxWarning: invalid escape sequence '\s'
<>:92: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_58289/290878098.py:92: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(filepath, sep='\s+', na_values='nan')


Iniciando análisis de experimentos...
Procesando archivos de control en ./experimentos/de 1000/...
  > Procesando control: 25-10-28 10-40 1000 17.txt
  > Procesando control: 25-10-28 10-55 1000 17.txt
  > Procesando control: 25-10-28 11-06 1000 17.txt
  > Procesando control: 25-10-28 11-20 1000 17.txt
Advertencia: Saltando archivo ./experimentos/de 1000/25-10-28 11-20 1000 17.txt - Faltan columnas 'Time' o 'CO2'.
  > Procesando control: 25-10-28 11-33 1000 17.txt
  > Procesando control: 25-10-28 11-50 1000 17.txt
  > Procesando control: 25-10-30 1003 19.txt
  > Procesando control: 25-10-30 1017 19 0.5 1.txt

Procesando archivos de experimento en ./experimentos/...
  > Procesando experimento: 25-10-30 1119 21.txt
  > Procesando experimento: 25-10-30 1141 21.txt
  > Procesando experimento: 25-10-30 1159 21 22.txt
  > Procesando experimento: 25-10-30 1215 21.txt
  > Procesando experimento: 25-10-30 1228 22.txt
  > Procesando experimento: 25-10-30 1241 22.txt
  > Procesando experimento: 25

/tmp/ipykernel_58289/290878098.py:380: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_anova = anova_table['PR(>F)'][0]
/tmp/ipykernel_58289/290878098.py:214: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='freq', y='effect_score', data=df, order=freq_order, palette='Set2')



Gráfico resumen guardado en: summary_boxplot.png
